<a href="https://colab.research.google.com/github/dgutiluns/water-anomaly-ca/blob/main/modeling_sample_and_streamcat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor, IsolationForest
from google.colab import drive

In [2]:
drive.mount('/content/drive')


path = '/content/drive/MyDrive/Year 3 Berk/Data Discovery Group Folder/sample_with_streamcat.csv'
df = pd.read_csv(path)
df.head()


Mounted at /content/drive


/tmp/ipython-input-2333323550.py:5: DtypeWarning: Columns (28,29,31) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


,_id,Program,ParentProject,Project,StationName,StationCode,SampleDate,CollectionTime,LocationCode,CollectionDepth,...,prism_norm_8110.precip8110ws,prism_norm_8110.tmean8110ws,prism_norm_8110.tmax8110ws,prism_norm_8110.tmin8110ws,wetness.wetindexws,wetness.wetindexcat,runoff.runoffws,runoff.runoffcat,landfire_veg.pctnonagintrodmanagvegws,landfire_veg.pctnonagintrodmanagvegcat
0,1,Proposition 84,Prop84-City of San Diego,City of San Diego Hypolimnetic Oxygenation System,HGA,HGA,2017-04-18T00:00:00,00:00,Not Recorded,0.0,...,477.242082,16.407631,24.216835,8.59334,687.8284,714.2622,38.9634,39.3049,16.61,1.61
1,2,Proposition 84,Prop84-City of San Diego,City of San Diego Hypolimnetic Oxygenation System,HGA,HGA,2017-04-18T00:00:00,00:00,Not Recorded,3.0,...,477.242082,16.407631,24.216835,8.59334,687.8284,714.2622,38.9634,39.3049,16.61,1.61
2,3,Proposition 84,Prop84-City of San Diego,City of San Diego Hypolimnetic Oxygenation System,HGA,HGA,2017-04-18T00:00:00,00:00,Not Recorded,6.0,...,477.242082,16.407631,24.216835,8.59334,687.8284,714.2622,38.9634,39.3049,16.61,1.61
3,4,Proposition 84,Prop84-City of San Diego,City of San Diego Hypolimnetic Oxygenation System,HGA,HGA,2017-04-18T00:00:00,00:00,Not Recorded,9.0,...,477.242082,16.407631,24.216835,8.59334,687.8284,714.2622,38.9634,39.3049,16.61,1.61
4,5,Proposition 84,Prop84-City of San Diego,City of San Diego Hypolimnetic Oxygenation System,HGA,HGA,2017-04-18T00:00:00,00:00,Not Recorded,12.0,...,477.242082,16.407631,24.216835,8.59334,687.8284,714.2622,38.9634,39.3049,16.61,1.61


In [5]:
# Work on a deep copy so the original df stays intact
df_proc = df.copy(deep=True)

# 1) Coerce numeric on the copy
num_cols = ["Result","RL","MDL","DilutionFactor","Latitude","Longitude"]
for c in num_cols:
    if c in df_proc.columns:
        df_proc[c] = pd.to_numeric(df_proc[c], errors="coerce")

# 2) Fill censored values (Result -> RL -> MDL)
mask_empty  = df_proc["Result"].isna()
mask_rl_bad = df_proc["RL"].isna() | (df_proc["RL"] < 0)

df_proc["Result_filled"] = df_proc["Result"]
df_proc.loc[mask_empty & ~mask_rl_bad, "Result_filled"] = df_proc.loc[mask_empty & ~mask_rl_bad, "RL"]
df_proc.loc[mask_empty &  mask_rl_bad, "Result_filled"] = df_proc.loc[mask_empty &  mask_rl_bad, "MDL"]

# Audit flag
df_proc["SubbedFrom"] = np.where(mask_empty & ~mask_rl_bad, "RL",
                         np.where(mask_empty & mask_rl_bad, "MDL", "Result"))

# 3) Normalize units to µg/L
def to_ugL(val, unit):
    if pd.isna(val): return np.nan
    u = (str(unit) if unit is not None else "").strip().lower()
    if u in ["µg/l","ug/l","micrograms per liter","microgram per liter"]:
        return float(val)
    if u in ["mg/l","milligrams per liter","milligram per liter"]:
        return float(val) * 1000.0
    if u in ["ng/l","nanograms per liter","nanogram per liter"]:
        return float(val) * 0.001
    # Unknown → pass through
    return float(val)

df_proc["Result_ugL"] = [to_ugL(v, u) for v, u in zip(df_proc["Result_filled"], df_proc.get("Unit", np.nan))]


,_id,Program,ParentProject,Project,StationName,StationCode,SampleDate,CollectionTime,LocationCode,CollectionDepth,...,prism_norm_8110.tmin8110ws,wetness.wetindexws,wetness.wetindexcat,runoff.runoffws,runoff.runoffcat,landfire_veg.pctnonagintrodmanagvegws,landfire_veg.pctnonagintrodmanagvegcat,Result_filled,SubbedFrom,Result_ugL
0,1,Proposition 84,Prop84-City of San Diego,City of San Diego Hypolimnetic Oxygenation System,HGA,HGA,2017-04-18T00:00:00,00:00,Not Recorded,0.0,...,8.593340,687.8284,714.2622,38.9634,39.3049,16.61,1.61,0.050,MDL,50.0
1,2,Proposition 84,Prop84-City of San Diego,City of San Diego Hypolimnetic Oxygenation System,HGA,HGA,2017-04-18T00:00:00,00:00,Not Recorded,3.0,...,8.593340,687.8284,714.2622,38.9634,39.3049,16.61,1.61,0.050,MDL,50.0
2,3,Proposition 84,Prop84-City of San Diego,City of San Diego Hypolimnetic Oxygenation System,HGA,HGA,2017-04-18T00:00:00,00:00,Not Recorded,6.0,...,8.593340,687.8284,714.2622,38.9634,39.3049,16.61,1.61,0.492,Result,492.0
3,4,Proposition 84,Prop84-City of San Diego,City of San Diego Hypolimnetic Oxygenation System,HGA,HGA,2017-04-18T00:00:00,00:00,Not Recorded,9.0,...,8.593340,687.8284,714.2622,38.9634,39.3049,16.61,1.61,0.662,Result,662.0
4,5,Proposition 84,Prop84-City of San Diego,City of San Diego Hypolimnetic Oxygenation System,HGA,HGA,2017-04-18T00:00:00,00:00,Not Recorded,12.0,...,8.593340,687.8284,714.2622,38.9634,39.3049,16.61,1.61,0.759,Result,759.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8610,49978,Total Maximum Daily Load,Nutrient TMDL for Santa Margarita Watershed,Nutrient TMDL for Santa Margarita Watershed,Pipe discharging to Brow Ditch on Huffstatler St.,HST02,2022-01-10T00:00:00,10:12,Not Recorded,-88.0,...,11.144219,678.6893,688.0435,28.8988,28.6356,2.50,4.69,59.000,Result,59.0
8611,49984,Total Maximum Daily Load,Nutrient TMDL for Santa Margarita Watershed,Nutrient TMDL for Santa Margarita Watershed,Rainbow Creek @ Old Highway 395,RBC04,2022-01-10T00:00:00,10:26,Not Recorded,-88.0,...,11.144219,678.6893,688.0435,28.8988,28.6356,2.50,4.69,1790.000,Result,1790000.0
8612,49987,Total Maximum Daily Load,Nutrient TMDL for Santa Margarita Watershed,Nutrient TMDL for Santa Margarita Watershed,Rainbow Creek @ Old Highway 395,RBC04,2022-01-10T00:00:00,10:26,Not Recorded,-88.0,...,11.144219,678.6893,688.0435,28.8988,28.6356,2.50,4.69,56.000,Result,56.0
8613,49993,Total Maximum Daily Load,Nutrient TMDL for Santa Margarita Watershed,Nutrient TMDL for Santa Margarita Watershed,Rainbow Glen Tributary to Rainbow Creek,RGT01,2022-01-10T00:00:00,10:55,Not Recorded,-88.0,...,11.585818,617.7526,617.7526,27.0000,27.0000,2.69,2.69,1250.000,Result,1250000.0


In [7]:
# Overall missingness table
summary = (
    pd.DataFrame({
        "column": df_proc.columns,
        "dtype": [str(df_proc[c].dtype) for c in df_proc.columns],
        "n_missing": [df_proc[c].isna().sum() for c in df_proc.columns],
    })
    .assign(n=len(df_proc))
)
summary["missing_rate"] = summary["n_missing"] / summary["n"]

# Target substitution audit
summary_target = pd.DataFrame({
    "total_rows": [len(df_proc)],
    "result_non_null": [df_proc["Result"].notna().sum()],
    "subbed_from_RL": [int((df_proc["SubbedFrom"]=="RL").sum())],
    "subbed_from_MDL": [int((df_proc["SubbedFrom"]=="MDL").sum())],
    "units_missing_in_target": [int(df_proc.loc[df_proc["Result"].isna(), "Unit"].isna().sum())]
})
summary.sort_values("missing_rate", ascending=False, inplace=True)

print("Target substitution audit:")
display(summary_target)
print("\nTop 30 columns by missing rate:")
display(summary.head(30))



Target substitution audit:


,total_rows,result_non_null,subbed_from_RL,subbed_from_MDL,units_missing_in_target
0,8615,7901,410,304,0



Top 30 columns by missing rate:


,column,dtype,n_missing,n,missing_rate
21,Observation,float64,8615,8615,1.000000
35,GroupSamples,float64,8615,8615,1.000000
53,StartingBank,float64,8615,8615,1.000000
55,UnitDistanceFromBank,float64,8615,8615,1.000000
54,DistanceFromBank,float64,8615,8615,1.000000
57,UnitStreamWidth,float64,8615,8615,1.000000
52,OccupationMethod,float64,8615,8615,1.000000
60,HydroMod,float64,8615,8615,1.000000
59,UnitStationWaterDepth,float64,8615,8615,1.000000
58,StationWaterDepth,float64,8615,8615,1.000000


In [8]:
# Columns that have any missingness
cols_with_missing = summary.loc[summary["n_missing"] > 0, "column"].tolist()

def miss_rate_by_group(df_like, group_col, cols):
    out = []
    for c in cols:
        mr = df_like.groupby(group_col)[c].apply(lambda s: s.isna().mean())
        out.append(mr.rename(c))
    return pd.concat(out, axis=1)

miss_analyte = miss_rate_by_group(df_proc, "Analyte", cols_with_missing)

# Worst offenders averaged across analytes
mean_miss_by_col = miss_analyte.mean(axis=0).sort_values(ascending=False)
print("Columns with highest average missing rate across analytes:")
display(mean_miss_by_col.head(20))

# Save wide table
# miss_analyte.to_csv("missingness_by_analyte.csv")
# print("Wrote missingness_by_analyte.csv")


Columns with highest average missing rate across analytes:


,0
Observation,1.000000
GroupSamples,1.000000
StartingBank,1.000000
UnitDistanceFromBank,1.000000
DistanceFromBank,1.000000
UnitStreamWidth,1.000000
OccupationMethod,1.000000
HydroMod,1.000000
UnitStationWaterDepth,1.000000
StationWaterDepth,1.000000


In [12]:
still_missing_target = df_proc["Result_ugL"].isna().values.sum(0)
still_missing_target

np.int64(0)

In [ ]:
# --- choose your target (optionally log-transform per analyte)
y = df_proc["Result_ugL"].to_numpy()
X = df_proc[feature_cols].to_numpy()

# 1) OOF predictions
kf = KFold(n_splits=5, shuffle=True, random_state=42)
oof_pred = np.zeros_like(y, dtype=float)

for tr, te in kf.split(X):
    model = RandomForestRegressor(n_estimators=500, n_jobs=-1, random_state=42)
    model.fit(X[tr], y[tr])
    oof_pred[te] = model.predict(X[te])

resid = y - oof_pred          # or np.log1p(y) - np.log1p(oof_pred)
abs_resid = np.abs(resid)

# 2) Isolation Forest on residuals
iso = IsolationForest(contamination=0.05, random_state=42)
labels = iso.fit_predict(abs_resid.reshape(-1, 1))   # -1 = outlier, 1 = inlier

df["residual"] = resid
df["if_label"] = labels
df["is_outlier"] = (labels == -1)
